In [ ]:
# 安装PaddleDetection
%cd E:/CAR/PaddleDetection
!pip install -r requirements.txt  
!python setup.py install

# 数据分析

In [ ]:
# 对官方数据集进行详细分析
# analyze.py——数据分析
# visualize.py——GT框可视化
%cd /home/aistudio/work/DataAnalyze
!python analyze.py voc /home/aistudio/Car2024/annotations/ --out ./out/

In [ ]:
# 生成训练集和验证集标签的id文件
# 随机划分数据集（可在代码中修改train_ratio的值从而改变训练集的划分比例）
!python /work/voc2coco/random_split_ids.py

In [ ]:
# voc转coco格式，得到train.json
!python /PaddleDetection/tools/x2coco.py \
    --dataset_type=voc \
    --voc_anno_dir='/Car2024/Annotations' \
    --voc_anno_list='/work/voc2coco/train_ann_ids.txt' \
    --voc_label_list='/Car2024/label_list.txt' \
    --output_dir='/Car2024/' \
    --voc_out_name=train.json
    

# voc转coco格式，得到val.json
!python /PaddleDetection/tools/x2coco.py \
    --dataset_type=voc \
    --voc_anno_dir='/Car2024/Annotations' \
    --voc_anno_list='/work/voc2coco/val_ann_ids.txt' \
    --voc_label_list='/Car2024/label_list.txt' \
    --output_dir='/Car2024/' \
    --voc_out_name=val.json


In [ ]:
# 分析id数据
!python /work/work/id.py

In [ ]:
# 根据数据集的特征数目来优化增强数据（使用copy-paste）
!python /work/copy_paste.py \
    --input_dir=/Car2024/Images \
    --json_path=/Car2024/train.json \
    --output_json=/Car2024/train_1.json \
    --muti_obj \
    --category_id=11 \
    --copypaste_ratio=0.45

!python /work/copy_paste.py \
    --input_dir=/Car2024/Images \
    --json_path=/Car2024/train_1.json \
    --output_json=/Car2024/train_2.json \
    --muti_obj \
    --category_id=12 \
    --copypaste_ratio=0.45

!python /work/copy_paste.py \
    --input_dir=/Car2024/Images \
    --json_path=/Car2024/train_2.json \
    --output_json=/Car2024/train_3.json \
    --muti_obj \
    --category_id=13 \
    --copypaste_ratio=0.45

!python /work/copy_paste.py \
    --input_dir=/Car2024/Images \
    --json_path=/Car2024/train_3.json \
    --output_json=/Car2024/train_4.json \
    --muti_obj \
    --category_id=5 \
    --copypaste_ratio=0.45

!python /work/copy_paste.py \
    --input_dir=/Car2024/Images \
    --json_path=/Car2024/train_4.json \
    --output_json=/Car2024/train_5.json \
    --muti_obj \
    --category_id=7 \
    --copypaste_ratio=0.45

!python /work/copy_paste.py \
    --input_dir=/Car2024/Images \
    --json_path=/Car2024/train_5.json \
    --output_json=/Car2024/train_6.json \
    --muti_obj \
    --category_id=1 \
    --copypaste_ratio=0.45


······

In [ ]:
# 查看修改之后的数据
]
import json
from IPython.display import display, JSON

# 查看增强后的JSON文件（示例）
# 选择一个增强后的JSON文件路径进行展示
enhanced_json_path = f'/home/aistudio/Car2024/train_4.json'  # 根据实际生成的文件路径修改

# 读取并展示增强后的JSON文件内容
with open(enhanced_json_path, 'r') as f:
    enhanced_data = json.load(f)

# 使用 IPython.display 显示 JSON 内容
display(JSON(enhanced_data))

In [ ]:
# 对标注类别id和数据统计
import json
from collections import defaultdict

def get_category_distribution(json_path):
    """
    获取JSON文件中每个类别的注释数量
    :param json_path: JSON文件路径
    :return: 类别ID到注释数量的映射字典
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    annotations = data['annotations']
    category_distribution = defaultdict(int)
    
    for annotation in annotations:
        category_id = annotation['category_id']
        category_distribution[category_id] += 1
    
    return category_distribution


json_paths = ['/Car2024/train.json', '/Car2024/train_10.json']

for json_path in json_paths:
    category_distribution = get_category_distribution(json_path)
    print(f"文件: {json_path}")
    for category_id, count in category_distribution.items():
        print(f"  类别ID: {category_id}, 注释数量: {count}")
    print("\n")

In [ ]:
# 输出json文件基本信息
!python E:/CAR/work/json_infoShow.py --json_path=E:/car/test/train.json

In [ ]:
# 这里的命令指的是替换配置文件中的数据集文件
!echo "romve the old dataset config"
!rm ~/PaddleDetection/configs/datasets/coco_detection.yml
!echo "copy new config to target dir"
!cp -p ~/coco_detection.yml ~/PaddleDetection/configs/datasets/
!echo "complete! is your show time please~"

In [ ]:
import paddle
print(paddle.device.is_compiled_with_cuda())


In [ ]:
use_cuda = True
place = paddle.CUDAPlace(0) if use_cuda else paddle.CPUPlace()


In [ ]:
# ppyoloe
%cd E:/car/PaddleDetection/
!python tools/train.py -c configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml --eval --use_vdl=true --vdl_log_dir='./output'

In [ ]:
# 删除多余文件，保留最后模型

%cd ~/PaddleDetection/output/cascade_rcnn_r50_fpn_1x_coco
!find . -type f -name '[0-9]'
!find . -type f -name '[0-9]*' -exec rm -f {} \;
!echo "delete checkpoints complete!"

# 预测结果

In [ ]:
# 预测模型
%cd ~
%cd PaddleDetection

!python tools/infer.py -c configs/picodet/picodet_m_320_coco_lcnet.yml \
                    --infer_img=/Car2024/images\
                    --output_dir=infer_output/ \
                    --draw_threshold=0.3 \
                    -o weights=/home/aistudio/PaddleDetection/output/picodet_m_320_coco_lcnet/model_final.pdparams \

In [ ]:
# 生成最终模型
%cd ~
%cd PaddleDetection
!python tools/export_model.py -c configs/yolov3/yolov3_r34_270e_coco.yml \
        -o weights=/home/aistudio/PaddleDetection/output/yolov3_r34_270e_coco/model_final \
        TestReader.fuse_normalize=true

In [ ]:
import os  
# 指定要搜索的文件夹路径  
folder_path = "/home/aistudio/Car2024/images"  
# 指定要写入的txt文件路径  
output_file = "/home/aistudio/submission/output.txt"
# 获取文件夹下的所有文件和子文件夹  
files = os.listdir(folder_path)
# 打开txt文件准备写入  
with open(output_file, "w") as file:  
    # 遍历所有文件和子文件夹  
    for item in files:  
        # 如果不是文件夹，则写入文件路径到txt文件中  
        if not os.path.isdir(os.path.join(folder_path, item)):  
            file.write(os.path.join(folder_path, item) + "\n")

In [ ]:
# 根据predict写对应的result.json
%cd ~/submission
!python predict.py output.txt result.json

In [ ]:
%cd ~/submission
!zip -r submission.zip .
!echo "please download the submission.zip!"